Hunter Bennett | Strains Project | 17 June 2021

In [17]:
### header ###
__author__ = "Hunter Bennett"
__license__ = "BSD"
__email__ = "hunter.r.bennett@gmail.com"
%load_ext autoreload
%autoreload 2
### imports ###
import sys
%matplotlib inline
import os
import re
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns

# import custom functions
import sys
sys.path.insert(0, '/home/h1bennet/code/')
from hbUtils import ngs_qc, quantile_normalize_df
from plotting_scripts import label_point, pca_rpkm_mat, get_diff_volcano
from homer_preprocessing import read_annotated_peaks, import_homer_diffpeak, pull_comparisons_get_diff

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#### PLOTTING PARAMETERS FOR MANUSCRIPT ####
# # get matplotlib to save readable fonts
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.size'] = 6
matplotlib.rcParams['savefig.dpi'] = 500

# line widths
matplotlib.rcParams['axes.linewidth'] = 1
matplotlib.rcParams['xtick.major.width'] = 1
matplotlib.rcParams['ytick.major.width'] = 1

# adjust defualt color for plots to black
# normal default is a dark gray
COLOR = 'black'
matplotlib.rcParams['text.color'] = COLOR
matplotlib.rcParams['axes.labelcolor'] = COLOR
matplotlib.rcParams['xtick.color'] = COLOR
matplotlib.rcParams['ytick.color'] = COLOR
matplotlib.rcParams['axes.edgecolor'] = COLOR

#### PLOT PARAMETERS FOR THIS PLOT ####

In [19]:
atacDirectory = '/home/h1bennet/strains/results/06_Strains_Control_Cohort2_ATAC/'
h3k27acDirectory = '/home/h1bennet/strains/results/06b_Strains_Control_Combined_H3K27Ac/'
workingDirectory = '/home/h1bennet/strains_manuscript/results/04_Strains_Manuscript_Direct_BALB_C57_Epigenetic_Comparison/'
if not os.path.isdir(workingDirectory):
    os.mkdir(workingDirectory)
os.chdir(workingDirectory)

In [20]:
if not os.path.isdir('./c57bl6j_balbcj_pairwise_epigenetics/'):
    os.mkdir('./c57bl6j_balbcj_pairwise_epigenetics/')

In [21]:
diff_peak, peaks, peak_mat, peak_mat_quant = import_homer_diffpeak(
    h3k27acDirectory+'/merged_peaks/diff_output.txt',
    h3k27acDirectory+'/merged_peaks/ann_norm_kc_control_atac_peaks_all.txt')

comp_dict = pull_comparisons_get_diff(diff_peak, seq_type='Peak')
comp_dict.keys()

annotatePeaks all peaks (86301, 27)
getDiffExpression selected transcripts (84264, 36)
annotatePeaks selected peaks (84264, 27)


dict_keys(['aj vs. balbcj', 'aj vs. c57bl6j', 'balbcj vs. c57bl6j'])

In [22]:
comps = ['balbcj vs. c57bl6j']
labels = ['BALB/cJ\nLog2(ATAC Tags)',
          'C57BL/6J\nLog2(ATAC Tags)a']

groups = [[[3,4,5],
         [6,7,8]]]

cols = []
for i in groups:
    for j in i:
        cols.extend(j)
cols = [cols]

colors = [['#3182bd', '#31a354']]
log2fc = np.log2(2)
pval = 0.05

In [23]:
for key, col, group, color in zip(comps, cols, groups, colors):
    
    # make columns for plotting
    de = comp_dict[key]
    de['logtpm'] = np.log2(peak_mat.iloc[:, col].mean(1)+1)
    de['log10p'] = -np.log10(de.adj_pval + 10**(-50))
    de['g0_mean'] = np.log2(peak_mat.iloc[:, group[0]].mean(1)+1)
    de['g1_mean'] = np.log2(peak_mat.iloc[:, group[1]].mean(1)+1)
    de = de.reindex(de.index[(de.g0_mean >= 3) | (de.g1_mean >= 3)])
    
    dot_colors = []
    dot_sizes = []
    
    # create dot size list
    for index, row in de.iterrows():
        if (row.adj_pval <= pval) & (-row.log2fc < -log2fc):
            dot_colors.append(color[1])
            dot_sizes.append(row.log10p)
        elif (row.adj_pval <= pval) & (-row.log2fc > log2fc):
            dot_colors.append(color[0])
            dot_sizes.append(row.log10p)
        else:
            dot_colors.append('#bdbdbd')
            dot_sizes.append(0.5)
    
    # print DE peaks
    print(key)
    print('N peaks downregulated', np.sum((de.adj_pval <= pval) & (de.log2fc < -log2fc)))
    print('N peaks upregulated', np.sum((de.adj_pval <= pval) & (de.log2fc > log2fc)))
    print('')
    
    fig, ax = plt.subplots(figsize=(1.5,1.5))
    
        # plot group by group scatter:
    ax.scatter(
        x=de.g0_mean,
        y=de.g1_mean,
        s=0.5, #de.log10p+0.05,
        c=dot_colors,
        rasterized=True)
    
    
    # set labels
    ax.set_xlabel(labels[0])
    ax.set_ylabel(labels[1])
    
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    #set max and min
    countmax = np.max([np.max(de.g0_mean), np.max(de.g1_mean)])
    ax.set_xlim([0,np.ceil(countmax)])
    ax.set_ylim([0,np.ceil(countmax)])
    
    ax.text(0.5,13,
        s='%i' % (np.sum((de.adj_pval <= 0.05) & (de.log2fc > 1))),
        fontsize=6,
        c='k')
    ax.text(13,0.5,
        s='%i' % (np.sum((de.adj_pval <= 0.05) & (de.log2fc < -1))),
        fontsize=6,
        c='k',
        rotation=0)
    
    plt.savefig('./c57bl6j_balbcj_pairwise_epigenetics/c57bl6j_balbcj_h3k27ac_scatter.pdf',
                bbox_inches='tight')
    plt.close()

balbcj vs. c57bl6j
N peaks downregulated 2474
N peaks upregulated 2397



# Now lets do ATAC

In [24]:
diff_peak, peaks, peak_mat, peak_mat_quant = import_homer_diffpeak(
    atacDirectory+'/merged_peaks/diff_output.txt',
    atacDirectory+'./merged_peaks/ann_norm_idr_peaks_merged.txt')

comp_dict = pull_comparisons_get_diff(diff_peak, seq_type='Peak')
comp_dict.keys()

annotatePeaks all peaks (86301, 30)
getDiffExpression selected transcripts (84264, 39)
annotatePeaks selected peaks (84264, 30)


dict_keys(['aj vs. balbcj', 'aj vs. c57bl6j', 'balbcj vs. c57bl6j'])

In [25]:
comps = ['balbcj vs. c57bl6j']
labels = ['BALB/cJ\nLog2(H3K27Ac Tags)',
          'C57BL/6J\nLog2(H3K27Ac Tags)']

groups = [[[4,5,6,7],
         [8,9,10,11]]]

cols = []
for i in groups:
    for j in i:
        cols.extend(j)
cols = [cols]

colors = [['#3182bd', '#31a354']]
log2fc = np.log2(2)
pval = 0.05

In [26]:
for key, col, group, color in zip(comps, cols, groups, colors):
    
    # make columns for plotting
    de = comp_dict[key]
    de['logtpm'] = np.log2(peak_mat.iloc[:, col].mean(1)+1)
    de['log10p'] = -np.log10(de.adj_pval + 10**(-50))
    de['g0_mean'] = np.log2(peak_mat.iloc[:, group[0]].mean(1)+1)
    de['g1_mean'] = np.log2(peak_mat.iloc[:, group[1]].mean(1)+1)
    de = de.reindex(de.index[(de.g0_mean >= 3) | (de.g1_mean >= 3)])
    
    dot_colors = []
    dot_sizes = []
    
    # create dot size list
    for index, row in de.iterrows():
        if (row.adj_pval <= pval) & (-row.log2fc < -log2fc):
            dot_colors.append(color[1])
            dot_sizes.append(row.log10p)
        elif (row.adj_pval <= pval) & (-row.log2fc > log2fc):
            dot_colors.append(color[0])
            dot_sizes.append(row.log10p)
        else:
            dot_colors.append('#bdbdbd')
            dot_sizes.append(0.5)
    
    # print DE peaks
    print(key)
    print('N peaks downregulated', np.sum((de.adj_pval <= pval) & (de.log2fc < -log2fc)))
    print('N peaks upregulated', np.sum((de.adj_pval <= pval) & (de.log2fc > log2fc)))
    print('')
    
    fig, ax = plt.subplots(figsize=(1.5,1.5))
    
        # plot group by group scatter:
    ax.scatter(
        x=de.g0_mean,
        y=de.g1_mean,
        s=0.5, #de.log10p+0.05,
        c=dot_colors,
        rasterized=True)
    
    
    # set labels
    ax.set_xlabel(labels[0])
    ax.set_ylabel(labels[1])
    
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    #set max and min
    countmax = np.max([np.max(de.g0_mean), np.max(de.g1_mean)])
    ax.set_xlim([0,np.ceil(countmax)])
    ax.set_ylim([0,np.ceil(countmax)])
    
    ax.text(0.5,13,
        s='%i' % (np.sum((de.adj_pval <= 0.05) & (de.log2fc > 1))),
        fontsize=6,
        c='k')
    ax.text(13,0.5,
        s='%i' % (np.sum((de.adj_pval <= 0.05) & (de.log2fc < -1))),
        fontsize=6,
        c='k',
        rotation=0)
    
    plt.savefig('./c57bl6j_balbcj_pairwise_epigenetics/c57bl6j_balbcj_atac_scatter.pdf',
                bbox_inches='tight')
    plt.close()

balbcj vs. c57bl6j
N peaks downregulated 3570
N peaks upregulated 2957

